In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [13]:
ds= pd.read_csv('ClusterMain.csv')

In [14]:
ds

,자치구명,층,임대면적,보증금,계약일,건축년도,임대건물명,경제활동인구,기준금리,전세가격비율,정책성향,아파트매매실거래지수,회사채수익률,소비자물가지수,통화량,광공업지수
0,용산구,1,59.61,11500,2022-01-03,1949,단독/다가구,27153,1.5,67.5,2,175.40,2.63,104.69,3634.3,4.2
1,동작구,1,58.80,30000,2022-01-03,1994,단독/다가구,27153,1.5,67.5,2,175.40,2.63,104.69,3634.3,4.2
2,강서구,10,156.88,78000,2022-01-03,2002,아파트,27153,1.5,67.5,2,175.40,2.63,104.69,3634.3,4.2
3,영등포구,16,121.53,83000,2022-01-03,2001,아파트,27153,1.5,67.5,2,175.40,2.63,104.69,3634.3,4.2
4,마포구,18,157.30,110000,2022-01-03,2003,아파트,27153,1.5,67.5,2,175.40,2.63,104.69,3634.3,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80016,용산구,5,101.95,40000,2022-05-06,1971,아파트,28310,0.5,65.0,1,175.55,3.63,107.56,3656.1,3.3
80017,동작구,3,38.12,36000,2022-05-06,2015,다세대/연립,28310,0.5,65.0,1,175.55,3.63,107.56,3656.1,3.3
80018,동작구,1,31.71,22050,2022-05-06,2014,다세대/연립,28310,0.5,65.0,1,175.55,3.63,107.56,3656.1,3.3
80019,강동구,10,15.94,13650,2022-05-06,2012,아파트,28310,0.5,65.0,1,175.55,3.63,107.56,3656.1,3.3


In [6]:
#범주형/연속형 변수 분리
cat_col = ['자치구명', '임대건물명', '정책성향']
con_col = ['층', '임대면적', '계약일', '건축년도', '경제활동인구', '기준금리', '전세가격비율', '아파트매매실거래지수', '회사채수익률', '소비자물가지수', '통화량', '광공업지수']

In [15]:
ds_raw = ds.copy() #원본 데이터 copy

In [16]:
#get_dummies()로 범주형 변수 원핫인코딩
ds = pd.get_dummies(ds, columns=cat_col, drop_first=True)

In [19]:
ds

,층,임대면적,보증금,계약일,건축년도,경제활동인구,기준금리,전세가격비율,아파트매매실거래지수,회사채수익률,...,자치구명_영등포구,자치구명_용산구,자치구명_은평구,자치구명_종로구,자치구명_중구,자치구명_중랑구,임대건물명_단독/다가구,임대건물명_아파트,임대건물명_오피스텔,정책성향_2
0,1,59.61,11500,2022-01-03,1949,27153,1.5,67.5,175.40,2.63,...,0,1,0,0,0,0,1,0,0,1
1,1,58.80,30000,2022-01-03,1994,27153,1.5,67.5,175.40,2.63,...,0,0,0,0,0,0,1,0,0,1
2,10,156.88,78000,2022-01-03,2002,27153,1.5,67.5,175.40,2.63,...,0,0,0,0,0,0,0,1,0,1
3,16,121.53,83000,2022-01-03,2001,27153,1.5,67.5,175.40,2.63,...,1,0,0,0,0,0,0,1,0,1
4,18,157.30,110000,2022-01-03,2003,27153,1.5,67.5,175.40,2.63,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80016,5,101.95,40000,2022-05-06,1971,28310,0.5,65.0,175.55,3.63,...,0,1,0,0,0,0,0,1,0,0
80017,3,38.12,36000,2022-05-06,2015,28310,0.5,65.0,175.55,3.63,...,0,0,0,0,0,0,0,0,0,0
80018,1,31.71,22050,2022-05-06,2014,28310,0.5,65.0,175.55,3.63,...,0,0,0,0,0,0,0,0,0,0
80019,10,15.94,13650,2022-05-06,2012,28310,0.5,65.0,175.55,3.63,...,0,0,0,0,0,0,0,1,0,0


In [27]:
#train, test, val 분리
X = ds.drop(columns='보증금')
y = ds[['보증금']]

In [31]:
X

,층,임대면적,계약일,건축년도,경제활동인구,기준금리,전세가격비율,아파트매매실거래지수,회사채수익률,소비자물가지수,...,자치구명_영등포구,자치구명_용산구,자치구명_은평구,자치구명_종로구,자치구명_중구,자치구명_중랑구,임대건물명_단독/다가구,임대건물명_아파트,임대건물명_오피스텔,정책성향_2
0,1,59.61,2022-01-03,1949,27153,1.5,67.5,175.40,2.63,104.69,...,0,1,0,0,0,0,1,0,0,1
1,1,58.80,2022-01-03,1994,27153,1.5,67.5,175.40,2.63,104.69,...,0,0,0,0,0,0,1,0,0,1
2,10,156.88,2022-01-03,2002,27153,1.5,67.5,175.40,2.63,104.69,...,0,0,0,0,0,0,0,1,0,1
3,16,121.53,2022-01-03,2001,27153,1.5,67.5,175.40,2.63,104.69,...,1,0,0,0,0,0,0,1,0,1
4,18,157.30,2022-01-03,2003,27153,1.5,67.5,175.40,2.63,104.69,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80016,5,101.95,2022-05-06,1971,28310,0.5,65.0,175.55,3.63,107.56,...,0,1,0,0,0,0,0,1,0,0
80017,3,38.12,2022-05-06,2015,28310,0.5,65.0,175.55,3.63,107.56,...,0,0,0,0,0,0,0,0,0,0
80018,1,31.71,2022-05-06,2014,28310,0.5,65.0,175.55,3.63,107.56,...,0,0,0,0,0,0,0,0,0,0
80019,10,15.94,2022-05-06,2012,28310,0.5,65.0,175.55,3.63,107.56,...,0,0,0,0,0,0,0,1,0,0


In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [44]:
X_train

,층,임대면적,계약일,건축년도,경제활동인구,기준금리,전세가격비율,아파트매매실거래지수,회사채수익률,소비자물가지수,...,자치구명_영등포구,자치구명_용산구,자치구명_은평구,자치구명_종로구,자치구명_중구,자치구명_중랑구,임대건물명_단독/다가구,임대건물명_아파트,임대건물명_오피스텔,정책성향_2
7097,1,46.36,2022-01-17,1993,28012,0.5,64.9,175.40,2.63,104.69,...,0,0,0,0,0,0,1,0,0,0
44761,15,29.53,2022-03-05,2004,28310,0.5,65.3,175.10,3.03,106.06,...,0,1,0,0,0,0,0,1,0,0
79970,2,84.41,2022-05-06,1989,28310,0.5,65.0,175.55,3.63,107.56,...,0,0,0,0,0,0,0,1,0,0
31098,1,18.00,2022-02-17,2012,28310,0.5,65.3,173.70,2.87,105.30,...,0,0,0,0,0,0,1,0,0,0
63905,11,75.51,2022-04-02,1986,28310,0.5,65.0,175.55,3.63,106.85,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,2,84.64,2022-01-17,2019,28012,0.5,65.1,175.40,2.63,104.69,...,0,0,0,0,0,0,0,1,0,0
54886,6,28.13,2022-03-20,2022,28310,0.5,65.0,175.10,3.03,106.06,...,0,0,0,0,0,0,0,0,0,0
76820,5,46.34,2022-04-28,2015,28310,0.5,65.0,175.55,3.63,106.85,...,0,0,0,0,0,0,0,0,0,0
860,16,17.57,2022-01-04,2019,27418,1.5,67.9,175.40,2.63,104.69,...,0,0,0,0,0,0,0,0,1,1


In [ ]:
#연속형 변수 스케일링
scaler = StandardScaler()
scaler.fit(X_train.loc[:,con_col])

X_train.loc[:,con_col] = scaler.transform(X_train.loc[:,con_col])
X_test.loc[:,con_col]  = scaler.transform(X_test.loc[:,con_col])
X_val.loc[:,con_col]   = scaler.transform(X_val.loc[:,con_col])

In [ ]:
import lightgbm as lgb

In [ ]:
lgb_clf = lgb.LGBMClassifier(num_leaves=31, objective='binary')
lgb_clf.fit(X_train, y_train)
y_pred = lgb_clf.predict(X_val)
accuracy_score(y_val, y_pred)

In [ ]:
파라미터는 dict에
param = {'num_leaves': 31, 'objective': 'binary', 'metric':'auc'}

#train data를 별도의 Dataset객체로 만들어야 함
train_data = lgb.Dataset(X_train, label=y_train.values)

#boosting을 몇 번 할건지 결정
num_round = 10 

#모델에 훈련
bst = lgb.train(param, train_data, num_round)

#예측
y_pred_val = bst.predict(X_val)
print(y_pred_val)

In [35]:
#확률값을 라벨값으로
y_pred_val = pd.Series(y_pred_val).apply(lambda x: 1 if x>=0.5 else 0) 

#성능평가
accuracy_score(y_val, y_pred_val)